# TensorflowTutorialMNIST_CNN


In [4]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np


In [47]:
def cnn_model_fn(features, labels, mode):
    tf.set_random_seed(1234)
    img_input = tf.reshape(features["train_X"], [-1,28,28,1])
    
    # Input layer - here as a conv2d layer #1
    conv1 = tf.layers.conv2d(inputs=img_input, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
    
    # Dropout layer #1
    dropout1 = tf.layers.dropout(inputs=conv1, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))
    
    # Conv2d layer #2
    conv2 = tf.layers.conv2d(inputs=img_input, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
    
    # Dropout layer #2
    dropout2 = tf.layers.dropout(inputs=conv1, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))
    
    # Dense layer
    dense = tf.layers.dense(inputs=dropout2, units=1024, activation=tf.nn.relu)
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense, units=10)
    
    
    # Generate predictions
    tf.argmax(input=logits)
    
    # Apply the softmax activation function
    tf.nn.softmax(logits, name="softmax_tensor")
    
    # Compile the predictions in a dict and return EstimatorSpec as object
    predictions = {
        'class': tf.arg_max(input=logits, dimension=1), 'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Training sgd
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation - accuracy metrics
    eval_metrice = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrice)

In [48]:
from tensorflow.keras.datasets.mnist import load_data
(train_X, train_y),(test_X, test_y) = load_data()

train_X = train_X/np.float(255)
test_X = test_X/np.float(255)

In [49]:
# Estimator
clf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="minist_convent_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'minist_convent_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12583ca90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [50]:
# Set up logging for predictions - softmax_tensor from above
tensor_to_log = {'probabilities': 'softmax_tensor'}

# Log after every 50 steps 
logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

In [51]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"train_X": train_X},
    y=train_y,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
clf.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.


ValueError: Rank mismatch: Rank of labels (received 1) should equal rank of logits minus 1 (received 4).

In [9]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [100, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode =
        = tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
#train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
#eval_labels = eval_labels.astype(np.int32)  # not required

In [11]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000213810886D8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [13]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[0.09769561 0.09242155 0.12242685 0.10349286 0.08833923 0.08377763
  0.10346124 0.10632138 0.0959845  0.10607913]
 [0.09077498 0.09780826 0.11222378 0.09875676 0.0866317  0.08999076
  0.10460069 0.10434891 0.1122424  0.10262185]
 [0.10098866 0.09973273 0.10662092 0.09765492 0.08236204 0.08510206
  0.10663666 0.1123535  0.10696173 0.10158674]
 [0.09277004 0.10203736 0.12036037 0.09777576 0.0924996  0.08914497
  0.09926867 0.10878316 0.1020197  0.09534036]
 [0.08421284 0.09203643 0.11746185 0.09150477 0.08513995 0.10005749
  0.1161961  0.11080511 0.10501176 0.09757362]
 [0.09434368 0.10813645 0.11698007 0.09225211 0.08942102 0.08319969
  0.0

INFO:tensorflow:loss = 2.3015337, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Loss for final step: 2.3015337.


In [14]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:loss = 2.2987185, step = 2


KeyboardInterrupt: 

In [15]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-08-13:40:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-08-13:40:32
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.1457, global_step = 1, loss = 2.2992885
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1: /tmp/mnist_convnet_model\model.ckpt-1
{'accuracy': 0.1457, 'loss': 2.2992885, 'global_step': 1}
